In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchsummary
import torchvision.models as models
import torch.quantization as quant

In [2]:
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=False, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=16)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
    
        # Define the first convolutional block
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        
        # Define the second convolutional block
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        
        # Define the third convolutional block
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=True)
        
        # Define the fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.bn4 = nn.BatchNorm1d(256)
        self.relu4 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        # First convolutional block
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # Second convolutional block
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # Third convolutional block
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # Flatten the tensor using reshape
        x = x.reshape(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.fc2(x)
        
        return x


In [5]:
model = Model()
model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU(inplace=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

In [6]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('size (KB) :',os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')

In [7]:
def train(model,dataloader, device):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  epochs = 15
  for epoch in range(epochs):
      running_loss = 0.0
      model.train()
      for i, data in enumerate(dataloader, 0):
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()
          model = model.to(device)
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()

      print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')
            

In [8]:
def test(model, dataloader, device):
  correct = 0
  total = 0
  model.eval()
  with torch.no_grad():
      for data in dataloader:
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          model = model.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
      print('Accuracy of the network on the 10000 test images: %d %%' % (
      100 * correct / total))

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [10]:
train(model, trainloader, device)

Epoch 1, Loss: 1.3346383207289458
Epoch 2, Loss: 0.9363199083701425
Epoch 3, Loss: 0.7746949427572968
Epoch 4, Loss: 0.6601980487480188
Epoch 5, Loss: 0.5639158821738589
Epoch 6, Loss: 0.47874491625582166
Epoch 7, Loss: 0.40480777021983394
Epoch 8, Loss: 0.34501832201505256
Epoch 9, Loss: 0.28844733858276206
Epoch 10, Loss: 0.2339246535335508
Epoch 11, Loss: 0.1827062566805145
Epoch 12, Loss: 0.14074820302464924
Epoch 13, Loss: 0.11046213199339254
Epoch 14, Loss: 0.0859584022251427
Epoch 15, Loss: 0.0662677745784983


In [11]:
test(model, testloader, device)

Accuracy of the network on the 10000 test images: 72 %


In [13]:
model.eval()
modules_to_fuse = [
    ['conv1', 'bn1', 'relu1'],
    ['conv2', 'bn2', 'relu2'],
    ['conv3', 'bn3', 'relu3']
]
model = torch.quantization.fuse_modules(model, modules_to_fuse)

In [14]:
model = torch.nn.Sequential(torch.quantization.QuantStub(), model, torch.quantization.DeQuantStub())

In [15]:
model.qconfig = torch.quantization.default_qconfig
torch.quantization.prepare(model, inplace=True)

Sequential(
  (0): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (1): Model(
    (conv1): ConvReLU2d(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
    (bn1): Identity()
    (relu1): Identity()
    (conv2): ConvReLU2d(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
    (bn2): Identity()
    (relu2): Identity()
    (conv3): ConvReLU2d(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
    )
    (bn3): Identity()
    (relu3): Identity()
    (fc1): Linear(
      in_features=2048, out_features=256, bias=True
      (activation_post_process)

In [16]:
test(model, testloader, device)

Accuracy of the network on the 10000 test images: 72 %


In [17]:
model

Sequential(
  (0): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=-1.0, max_val=1.0)
  )
  (1): Model(
    (conv1): ConvReLU2d(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (activation_post_process): MinMaxObserver(min_val=0.0, max_val=13.884560585021973)
    )
    (bn1): Identity()
    (relu1): Identity()
    (conv2): ConvReLU2d(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (activation_post_process): MinMaxObserver(min_val=0.0, max_val=11.213496208190918)
    )
    (bn2): Identity()
    (relu2): Identity()
    (conv3): ConvReLU2d(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (activation_post_process): MinMaxObserver(min_val=0.0, max_val=9.220218658447266)
    )
    (bn3): Identity()
    (relu3): Identity()
    (fc1): Linear(
      in_features=2048, out_features=256, 

In [18]:
torch.quantization.convert(model, inplace=True)

Sequential(
  (0): Quantize(scale=tensor([0.0157]), zero_point=tensor([64]), dtype=torch.quint8)
  (1): Model(
    (conv1): QuantizedConvReLU2d(3, 32, kernel_size=(3, 3), stride=(1, 1), scale=0.10932724922895432, zero_point=0, padding=(1, 1))
    (bn1): Identity()
    (relu1): Identity()
    (conv2): QuantizedConvReLU2d(32, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.08829524368047714, zero_point=0, padding=(1, 1))
    (bn2): Identity()
    (relu2): Identity()
    (conv3): QuantizedConvReLU2d(64, 128, kernel_size=(3, 3), stride=(1, 1), scale=0.07260014861822128, zero_point=0, padding=(1, 1))
    (bn3): Identity()
    (relu3): Identity()
    (fc1): QuantizedLinear(in_features=2048, out_features=256, scale=0.061604488641023636, zero_point=70, qscheme=torch.per_tensor_affine)
    (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu4): ReLU(inplace=True)
    (fc2): QuantizedLinear(in_features=256, out_features=10, scale=0.3346237540245056, ze

In [19]:
test(model, testloader, device)

NotImplementedError: Could not run 'aten::native_batch_norm' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::native_batch_norm' is only available for these backends: [CPU, Meta, MkldnnCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnaps

In [4]:
model = torch.load("class.pth")

AttributeError: Can't get attribute 'ResNet' on <module '__main__'>